# 🔐 Cybersecurity Threat Detection Using AI - GUI Version (Tkinter)

This notebook allows you to:
- Train a Random Forest model on the KDD Cup 1999 dataset
- Launch a GUI to upload CSV files and detect attacks
- Save predictions to a new CSV file

In [ ]:
# ✅ Step 1: Install required libraries
!pip install pandas scikit-learn matplotlib seaborn

In [ ]:
# ✅ Step 2: Import Libraries
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# ✅ Step 3: Train the Model
def train_model():
    df = pd.read_csv("kddcup.csv", nrows=10000)  # Change to full for large training

    for col in df.select_dtypes(include=['object']).columns:
        df[col] = LabelEncoder().fit_transform(df[col])

    X = df.drop("label", axis=1)
    y = df["label"].apply(lambda x: 0 if x == 'normal' else 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    acc = accuracy_score(y_test, model.predict(X_test))
    return model, acc

model, accuracy = train_model()
print(f"✅ Model Trained | Accuracy: {accuracy*100:.2f}%")

In [ ]:
# ✅ Step 4: Create GUI to Upload CSV and Predict
def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if not file_path:
        return

    try:
        df = pd.read_csv(file_path)
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = LabelEncoder().fit_transform(df[col])
        predictions = model.predict(df)
        result = pd.DataFrame({'Prediction': predictions})
        result['Prediction'] = result['Prediction'].apply(lambda x: "Normal" if x == 0 else "Attack")
        result_file = "predicted_results.csv"
        result.to_csv(result_file, index=False)
        messagebox.showinfo("Success", f"Predictions saved to {result_file}")
    except Exception as e:
        messagebox.showerror("Error", str(e))

app = tk.Tk()
app.title("Cybersecurity Threat Detection using AI")
app.geometry("400x200")

label = tk.Label(app, text=f"Model Trained - Accuracy: {accuracy*100:.2f}%", font=("Arial", 12))
label.pack(pady=10)

btn = tk.Button(app, text="Upload CSV to Detect Threats", command=browse_file, font=("Arial", 12))
btn.pack(pady=20)

app.mainloop()